<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Google-Cloud-Speech-to-Text-API-Microphone-Streaming-emulate" data-toc-modified-id="Google-Cloud-Speech-to-Text-API-Microphone-Streaming-emulate-1">Google Cloud Speech-to-Text API Microphone Streaming emulate</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Microphone-Streaming-emulate" data-toc-modified-id="Microphone-Streaming-emulate-1.0.1">Microphone Streaming emulate</a></span></li><li><span><a href="#위의-기능을-Python-module로-작성한-기능-테스트" data-toc-modified-id="위의-기능을-Python-module로-작성한-기능-테스트-1.0.2">위의 기능을 Python module로 작성한 기능 테스트</a></span></li></ul></li></ul></li></ul></div>

# Google Cloud Speech-to-Text API Microphone Streaming emulate

- Microphone 오디오 파일을 Streaming 방식으로 처리하는 샘플(실제는 저장된 오디오 파일을 읽어서 Streaming 방식으로 처리)
- https://cloud.google.com/speech-to-text/docs/streaming-recognize?hl=ko#speech-streaming-mic-recognize-python
- **Microphone Streaming 오디오 처리는 300초 시간 제한이 있음** : 더 긴 파일은 파일을 잘라서 처리하는 방법을 사용할 수 있음<br>
   (OutOfRange: 400 Exceeded maximum allowed stream duration of 305 seconds.)

###  Microphone Streaming emulate

In [1]:
from __future__ import division

import sys
import re
import wave

from google.cloud import speech
from google.cloud.speech import enums
from google.cloud.speech import types

import pyaudio
from six.moves import queue

# Audio recording parameters
RATE = 16000
CHUNK = int(RATE / 10)  # 100ms

In [2]:
class MicrophoneStream(object):
    """Opens a recording stream as a generator yielding the audio chunks."""
    def __init__(self, rate, chunk):
        self._rate = rate
        self._chunk = chunk

        # Create a thread-safe buffer of audio data
        self._buff = queue.Queue()
        self.closed = True

    def __enter__(self):
        self._audio_interface = pyaudio.PyAudio()
        self._audio_stream = self._audio_interface.open(
            format=pyaudio.paInt16,
            # The API currently only supports 1-channel (mono) audio
            # https://goo.gl/z757pE
            channels=1, rate=self._rate,
            input=True, frames_per_buffer=self._chunk,
            # Run the audio stream asynchronously to fill the buffer object.
            # This is necessary so that the input device's buffer doesn't
            # overflow while the calling thread makes network requests, etc.
            stream_callback=self._fill_buffer,
        )

        self.closed = False

        return self

    def __exit__(self, type, value, traceback):
        self._audio_stream.stop_stream()
        self._audio_stream.close()
        self.closed = True
        # Signal the generator to terminate so that the client's
        # streaming_recognize method will not block the process termination.
        self._buff.put(None)
        self._audio_interface.terminate()

    def _fill_buffer(self, in_data, frame_count, time_info, status_flags):
        """Continuously collect data from the audio stream, into the buffer."""
#         self._buff.put(in_data)
        data = wf.readframes(frame_count)
        self._buff.put(data)
        stream.write(data)
        return None, pyaudio.paContinue

    def generator(self):
        while not self.closed:
            # Use a blocking get() to ensure there's at least one chunk of
            # data, and stop iteration if the chunk is None, indicating the
            # end of the audio stream.
            chunk = self._buff.get()
            if chunk is None:
                return
            data = [chunk]

            # Now consume whatever other data's still buffered.
            while True:
                try:
                    chunk = self._buff.get(block=False)
                    if chunk is None:
                        return
                    data.append(chunk)
                except queue.Empty:
                    break

            yield b''.join(data)

In [3]:
def listen_print_loop(responses):
    """Iterates through server responses and prints them.

    The responses passed is a generator that will block until a response
    is provided by the server.

    Each response may contain multiple results, and each result may contain
    multiple alternatives; for details, see https://goo.gl/tjCPAU.  Here we
    print only the transcription for the top alternative of the top result.

    In this case, responses are provided for interim results as well. If the
    response is an interim one, print a line feed at the end of it, to allow
    the next result to overwrite it, until the response is a final one. For the
    final one, print a newline to preserve the finalized transcription.
    """
    num_chars_printed = 0
    for response in responses:
        if not response.results:
            continue

        # The `results` list is consecutive. For streaming, we only care about
        # the first result being considered, since once it's `is_final`, it
        # moves on to considering the next utterance.
        result = response.results[0]
        if not result.alternatives:
            continue

        # Display the transcription of the top alternative.
        transcript = result.alternatives[0].transcript

        # Display interim results, but with a carriage return at the end of the
        # line, so subsequent lines will overwrite them.
        #
        # If the previous result was longer than this one, we need to print
        # some extra spaces to overwrite the previous result
        overwrite_chars = ' ' * (num_chars_printed - len(transcript))

        if not result.is_final:
            sys.stdout.write(transcript + overwrite_chars + '\r')
            sys.stdout.flush()

            num_chars_printed = len(transcript)

        else:
            print(transcript + overwrite_chars)

            # Exit recognition if any of the transcribed phrases could be
            # one of our keywords.
            if re.search(r'\b(exit|quit)\b', transcript, re.I):
                print('Exiting..')
                break

            num_chars_printed = 0

In [4]:
def microphone_streaming_start():
    # See http://g.co/cloud/speech/docs/languages
    # for a list of supported languages.
    language_code = 'ko-KR'

    client = speech.SpeechClient()
    config = types.RecognitionConfig(
        encoding=enums.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=RATE,
        language_code=language_code,
        enable_automatic_punctuation=True,
        enable_word_time_offsets=True)
    #     enable_speaker_diarization=True,
    #     diarization_speaker_count=3)

    streaming_config = types.StreamingRecognitionConfig(
        config=config,
        interim_results=True)

    with MicrophoneStream(RATE, CHUNK) as stream:
        audio_generator = stream.generator()
        requests = (types.StreamingRecognizeRequest(audio_content=content)
                    for content in audio_generator)

        responses = client.streaming_recognize(streaming_config, requests)

        # Now, put the transcription responses to use.
        listen_print_loop(responses)

In [5]:
FILE_NAME = "19m_20181220_deominju_16000.wav"
wf = wave.open("audio_file/"+ FILE_NAME, 'rb')     # 300초 이상의 파일은 분할해서 처리 해야 함

p = pyaudio.PyAudio()

stream = p.open(format=p.get_format_from_width(wf.getsampwidth()),
                channels=wf.getnchannels(),
                rate=wf.getframerate(),
                output=True)

In [6]:
# 중단된 경우 재실행 하면 중단된 부분부터 시작됨
try:
    microphone_streaming_start()
except KeyboardInterrupt:
    pass

In [7]:
stream.stop_stream()
stream.close()
p.terminate()

### 위의 기능을 Python module로 작성한 기능 테스트

In [8]:
from modules import google_stt as stt

In [9]:
import wave
import pyaudio

In [10]:
FILE_NAME = "19m_20181220_deominju_16000.wav"
wf = wave.open("audio_file/"+ FILE_NAME, 'rb')     # 300초 이상의 파일은 분할해서 처리 해야 함

p = pyaudio.PyAudio()

stream = p.open(format=p.get_format_from_width(wf.getsampwidth()),
                channels=wf.getnchannels(),
                rate=wf.getframerate(),
                output=True)

In [11]:
try:
    stt.microphone_streaming_start(wf, stream)    # 함수 호출을 위해 객체 전달
except KeyboardInterrupt:
    pass

네, 반갑습니다. 제 115차 정책조정회의 일을 홍영표 원내대표에 모드 바람으로 11 12 월 이렇게 보내기가. 아, 이제 일주일 앞으로 다가왔습니다. 여야가 민생법안 처리를 해서 노니의 속도를 더 내야겠습니다. 꼭 처리해야 할 일이 토파즈는 6300번 산업안전보건법입니다. 6003번 오늘 일찍 의견에 법안소위에서 노니 될 예정입니다. 오늘 합의가 이루어지지 않을 경우 27분의 철인가 사실상 어렵습니다. 자유한국당 너 우리 아이들을 위한은 마음이 조금이라도 있다면 일 1003번 노니 접수 임해야 합니다. 지난번 정기국회 때처럼 법안처리를
지연 시키려는 것은 국민이 용납하지 않을 것입니다. 우리당은 오늘 처리될 수 있도록 최선을 다해서 열린 자세로 보관소에 임하겠습니다. 그러나 끝내. 자유한국당이 반대한다면 테스트 트랙을 통해 법안 처리 하는 방안을 추진하겠습니다. 위험의 외주화를 말을 산업 안전 보건법 또 반드시 처리하겠습니다. 하청노동자들의 더 이상 죽음이 내몰리지 않기 위해서 필요한 법안입니다. 다행히 어제 황윤희 고용노동소위 해서 27일 본회의에서 산업안전보건법을 처리하기로 여야간 의견이 모아졌습니다. 앞으로 논의과정에서 차질없이 처리될 수 있도록 기도합니다.


In [12]:
stream.stop_stream()
stream.close()
p.terminate()